In [1]:
import os
import numpy as np
import glob

from train import Trainer, GAN_Trainer
import torch.nn as nn

from torch.utils.data import DataLoader
from tqdm import tqdm

from UNet_model import UNet
from UNet_GAN_model import UNetGen, UNetDis
from GAN_model import NetGen, NetDis
from ResNet_model import ResNet

from colorize_data import ColorizeData


In [2]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
paths = np.array([])
training_set_path = os.path.join(current_directory, 'Dataset', 'training_set')
for p in os.listdir(training_set_path):
    new_path = os.path.join(training_set_path,p)
    n_paths = np.array(glob.glob( new_path+ "/*.jpg"))
    paths = np.concatenate([paths,n_paths])

val_paths = np.array([])
validation_set_path = os.path.join(current_directory, 'Dataset', 'validation_set')
for p in os.listdir(validation_set_path):
    new_path = os.path.join(validation_set_path,p)
    n_paths = np.array(glob.glob( new_path+ "/*.jpg"))
    val_paths = np.concatenate([paths,n_paths])

train_indices = np.random.permutation(len(paths))
train_paths = paths[train_indices]
val_indices = np.random.permutation(len(val_paths))
val_paths = val_paths[val_indices]

# trainer = GAN_Trainer(train_paths, val_paths, epochs = 1, batch_size = 64, learning_rate = 0.01, num_workers = 2)
# trainer.train(NetG=UNetGen(), NetD=UNetDis())

In [3]:
trainers = { Trainer:[[ResNet(), 'ResNet'], [UNet(), 'UNet']],GAN_Trainer:[[NetGen(),NetDis(),"GAN"],[UNetGen(),UNetDis(), "UNetGAN"]]}
Losses = {}

for t in trainers.keys():
    for n in trainers[t]:
        trainer = t(train_paths, val_paths, epochs = 10, batch_size = 16, learning_rate = 0.01, num_workers = 2)
        loss, val_loss = trainer.train(*n)
        Losses[n[-1]]= [loss, val_loss]


Starting Training Epoch 1


100%|██████████| 195/195 [14:29<00:00,  4.46s/it]


Epoch 1 		 Training Loss: 0.290220847649452


In [4]:
Losses

{'ResNet': [[0.8162988026936849,
   0.5508615473906199,
   0.3843180040518443,
   0.3461948136488597,
   0.3016207814216614],
  [8370.706176757812,
   1105.971206665039,
   34.883737087249756,
   6.462254047393799,
   2.54803603887558]],
 'UNet': [[0.6264670590559641,
   0.4051589369773865,
   0.3544735610485077,
   0.2899746497472127,
   0.2556848078966141],
  [1.0973852574825287,
   1.031692087650299,
   0.8299153596162796,
   0.6190952807664871,
   0.3958797827363014]],
 'GAN': [[[0.6193176110585531, 13.864546457926432],
   [33.333333333333336, 31.934542338053387],
   [33.333333333333336, 13.763161977132162],
   [33.333333333333336, 13.780946095784506],
   [33.333333333333336, 13.586840311686197]],
  [[0.7420367399851481, 33.42511240641276],
   [33.333333333333336, 30.811915079752605],
   [33.333333333333336, 14.050852457682291],
   [33.333333333333336, 13.096725463867188],
   [33.333333333333336, 12.958033243815104]]],
 'UNetGAN': [[[0.7325641314188639, 8.508634567260742],
   [33.3

In [ ]:
import json

with open('losses.json', 'w') as f:
    json.dump(Losses, f, indent=4)